# Transfer Learning VGG16+Mobile Net

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import keras

#from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import pylab
#plt.rcParams['figure.figsize'] = 10, 10
#%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../ShipIceberg/Data"]).decode("utf8"))

Using TensorFlow backend.


sample_submission.csv
test.json
train.json



In [2]:
path = "../ShipIceberg/Data"
train = pd.read_json(path+"/train.json")
target_train=train['is_iceberg']
# test = pd.read_json(path+"/test.json")
import json
with open('../ShipIceberg/Data/test.json', 'r') as f:
    test = json.load(f)
    test=pd.DataFrame(test)
    
print('done!')

target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

done!


In [3]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

'''
# 需要循环？？？
# let's see if despeckling has any influence on the images
# plot nine different instances with different windows and noise levels (band 1)
windows = [2, 4, 8] # can be tuple too if not symetric
noise_var = np.array([1, 2, 4])

# 0-2
Wi = 0
Nj = 0

for i in range(0,1604):
    band_1_linear = decibel_to_linear(X_band_1[i])
    band_2_linear = decibel_to_linear(X_band_2[i])

    noise_var_1 = np.round(np.var(band_1_linear)*noise_var,10)
    noise_var_2 = np.round(np.var(band_2_linear)*noise_var,10)

    X_band_1[i] = linear_to_decibel(lee_filter(band_1_linear, windows[Wi], noise_var_1[Nj]))
    X_band_2[i] = linear_to_decibel(lee_filter(band_2_linear, windows[Wi], noise_var_2[Nj]))


# Sobel Operator & Averaging
from scipy import signal

Gx_sobel=np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
Gy_sobel=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
lpf=1.0/25*np.ones((5,5))

for i in range(0,1604):
    
    img_1=X_band_1[i].copy()
    img_lpf_1=signal.convolve2d(img_1,lpf,mode='same')
    img_Gx_1=signal.convolve2d(img_lpf_1,Gx_sobel,mode='same')
    img_Gy_1=signal.convolve2d(img_lpf_1,Gy_sobel,mode='same')

    img_2=X_band_2[i].copy()
    img_lpf_2=signal.convolve2d(img_2,lpf,mode='same')
    img_Gx_2=signal.convolve2d(img_lpf_2,Gx_sobel,mode='same')
    img_Gy_2=signal.convolve2d(img_lpf_2,Gy_sobel,mode='same')

    X_band_1[i] = np.hypot(img_Gx_1,img_Gy_1)
    X_band_2[i] = np.hypot(img_Gx_2,img_Gy_2)
'''

X_band_3 = np.zeros((1604,75,75))
for i in range(0,1604):
    subt = abs(X_band_1[i]-X_band_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_3[i]=W1 * X_band_1[i]+W2 * X_band_2[i]


# X_band_3=(X_band_1+X_band_2)/2  # 算术平均：尝试其他比例？
#X_band_3=X_band_1 * 0.5858 + X_band_2 * 0.4142
# 效果极差 X_band_3 = X_band_2 * X_band_2 / X_band_1
#X_band_3 = np.sqrt(X_band_1 * X_band_2)  # 调和·平均: sqrt invalid number???
#X_band_3 = X_band_1 / X_band_2
# X_band_3=X_band_2 * 2 - X_band_1

#from scipy import signal
#X_band_3=signal.fftconvolve(X_band_1, X_band_2, mode = 'same')

#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])

X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)


#Generate the test data
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])

'''
for i in range(0,8424):
    band_test_1_linear = decibel_to_linear(X_band_test_1[i])
    band_test_2_linear = decibel_to_linear(X_band_test_2[i])

    noise_var_test_1 = np.round(np.var(band_test_1_linear)*noise_var,10)
    noise_var_test_2 = np.round(np.var(band_test_2_linear)*noise_var,10)

    X_band_test_1[i] = linear_to_decibel(lee_filter(band_test_1_linear, windows[Wi], noise_var_test_1[Nj]))
    X_band_test_2[i] = linear_to_decibel(lee_filter(band_test_2_linear, windows[Wi], noise_var_test_2[Nj]))


for i in range(0,8424):
    
    img_1=X_band_test_1[i].copy()
    img_lpf_1=signal.convolve2d(img_1,lpf,mode='same')
    img_Gx_1=signal.convolve2d(img_lpf_1,Gx_sobel,mode='same')
    img_Gy_1=signal.convolve2d(img_lpf_1,Gy_sobel,mode='same')

    img_2=X_band_test_2[i].copy()
    img_lpf_2=signal.convolve2d(img_2,lpf,mode='same')
    img_Gx_2=signal.convolve2d(img_lpf_2,Gx_sobel,mode='same')
    img_Gy_2=signal.convolve2d(img_lpf_2,Gy_sobel,mode='same')

    X_band_test_1[i] = np.hypot(img_Gx_1,img_Gy_1)
    X_band_test_2[i] = np.hypot(img_Gx_2,img_Gy_2)
'''

X_band_test_3 = np.zeros((8424,75,75))
for i in range(0,8424):
    subt = abs(X_band_test_1[i]-X_band_test_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_test_3[i]=W1 * X_band_test_1[i]+W2 * X_band_test_2[i]

# X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=X_band_test_1 * 0.5858 + X_band_test_2 * 0.4142
#X_band_test_3 = X_band_test_2 * X_band_test_2 / X_band_test_1
#X_band_test_3 = np.sqrt(X_band_test_1*X_band_test_2)
#X_band_test_3=signal.fftconvolve(X_band_test_1, X_band_test_2, mode = 'same')
#X_band_test_3 = X_band_test_1 / X_band_test_2
# X_band_test_3=X_band_test_2 * 2 - X_band_test_1

#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])

X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

# del train;del test;

In [4]:
#Import Keras.
#from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=64

In [5]:
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   #es = EarlyStopping('val_loss', patience=10, mode="min")
   es = EarlyStopping('val_loss', patience=20, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]

In [6]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    x = GlobalMaxPooling2D()(x)
    base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

    x2 = base_model2.output
    x2 = GlobalAveragePooling2D()(x2)

    merge_one = concatenate([x, x2, angle_layer])
#     merge_one = concatenate([x2, angle_layer])

    merge_one = Dropout(0.6)(merge_one)
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [7]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "./model/%s_aug_mobile_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=10)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [8]:
preds=myAngleCV(X_train, X_angle, X_test)
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('subMobile.csv', index=False)


===================FOLD= 0


/home/jayden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - 25s - loss: 0.7076 - acc: 0.6151 - val_loss: 0.5627 - val_acc: 0.6729
Epoch 2/100
24/24 [==============================] - 14s - loss: 0.4351 - acc: 0.7875 - val_loss: 0.3182 - val_acc: 0.8561
Epoch 3/100
24/24 [==============================] - 14s - loss: 0.3713 - acc: 0.8203 - val_loss: 0.2922 - val_acc: 0.8486
Epoch 4/100
24/24 [==============================] - 15s - loss: 0.3234 - acc: 0.8515 - val_loss: 0.2524 - val_acc: 0.8766
Epoch 5/100
24/24 [==============================] - 14s - loss: 0.2922 - acc: 0.8552 - val_loss: 0.2315 - val_acc: 0.8822
Epoch 6/100
24/24 [==============================] - 14s - loss: 0.2778 - acc: 0.8782 - val_loss: 0.2098 - val_acc: 0.9047
Epoch 7/100
24/24 [==============================] - 14s - loss: 0.2339 - acc: 0.8988 - val_loss: 0.2036 - val_acc: 0.9140
Epoch 8/100
24/24 [==============================] - 14s - loss: 0.2531 - acc: 0.8949 - val_loss: 0.2161 - val_acc: 0.9103
Epoch 9/100
24/2

24/24 [==============================] - 14s - loss: 0.1507 - acc: 0.9368 - val_loss: 0.1905 - val_acc: 0.9383
Epoch 26/100
24/24 [==============================] - 14s - loss: 0.1461 - acc: 0.9389 - val_loss: 0.2432 - val_acc: 0.9308
Epoch 27/100
24/24 [==============================] - 14s - loss: 0.1569 - acc: 0.9306 - val_loss: 0.2329 - val_acc: 0.9234
Epoch 28/100
24/24 [==============================] - 14s - loss: 0.1613 - acc: 0.9367 - val_loss: 0.2942 - val_acc: 0.9121
Epoch 29/100
24/24 [==============================] - 14s - loss: 0.1576 - acc: 0.9319 - val_loss: 0.2546 - val_acc: 0.9234
Epoch 30/100
24/24 [==============================] - 14s - loss: 0.1505 - acc: 0.9385 - val_loss: 0.2029 - val_acc: 0.9346
Epoch 31/100
24/24 [==============================] - 14s - loss: 0.1318 - acc: 0.9461 - val_loss: 0.2039 - val_acc: 0.9290
Epoch 32/100
24/24 [==============================] - 14s - loss: 0.1287 - acc: 0.9443 - val_loss: 0.2389 - val_acc: 0.9364
Epoch 33/100
24/24 [=